<a href="https://colab.research.google.com/github/YangJiao85/PANDA-kaggle/blob/master/panda_png_kfold_1epoch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PANDA: start from tiles in png zip file and evaluate using KFold cross validation

This notebook got inspirations from
- [PANDA 16X128X128 tiles from Iafoss](https://www.kaggle.com/iafoss/panda-16x128x128-tiles) 

Thank you for sharing!

This note book extracts tiles from original images and save them as dataset panda-tile-ntxnrxnr.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Set some parameters.

In [2]:
""" PATH on kaggle
DATA_PATH     = '/kaggle/input/prostate-cancer-grade-assessment/'
TRAIN_PATH    = DATA_PATH + 'train_images/'
MASKS_PATH    = DATA_PATH + 'train_label_masks/'
TEST_PATH     = DATA_PATH + 'test_images/'
PP_PATH       = '/kaggle/input/panda-tiles/'
TRAIN_TILES_PATH = PP_PATH + 'train_images_tiles16x128x128/'
MASKS_TILES_PATH = PP_PATH + 'train_masks_tiles16x128x128/'
MODEL_PATH    = './'                                       # path to save model
ENET_MODEL_PATH = '/kaggle/input/efficientnettf/'          # pretrained model from efficientnet package
restart_path  = '/kaggle/input/panda-tiles-tf-keras-cohen-kappa-loss-baseline/model_v1.1.h5'
ZIP_TRAIN     = TRAIN_TILES_PATH + 'train.zip'                # zip file with tiled images
ZIP_MASKS     = MASKS_TILES_PATH + 'masks.zip'                # zip file with tiled masks
"""
""" PATH on Colab """
Colab_ROOT    = '/content/drive/My Drive/Colab Notebooks/competetions/'
DATA_PATH     = Colab_ROOT + 'input/prostate-cancer-grade-assessment/'
TRAIN_PATH    = DATA_PATH + 'train_images/'
MASKS_PATH    = DATA_PATH + 'train_label_masks/'
TEST_PATH     = DATA_PATH + 'test_images/'
TRAIN_TILES_PATH = DATA_PATH + 'train_images_tiles16x128x128/'
MASKS_TILES_PATH = DATA_PATH + 'train_masks_tiles16x128x128/'
MODEL_PATH    = Colab_ROOT + 'PANDA/'                         # path to save model
ENET_MODEL_PATH = './'                                        # pretrained model from efficientnet package
restart_path  = MODEL_PATH + 'model_v1.1.h5'
ZIP_TRAIN     = TRAIN_TILES_PATH + 'train.zip'                # zip file with tiled images
ZIP_MASKS     = MASKS_TILES_PATH + 'masks.zip'                # zip file with tiled masks


MODEL_VERSION = 'v1.2'                                     # version for the model to be saved
restart       = True
EPOCHS        = 1
TILE_SIZE     = 128      # 16 128x128 tiles are selected from each image and mask
NUM_TILE      = 16
BATCH_SIZE    = 4
INI_LR        = 1.e-4

SEED          = 2020
N_FOLD        = 4

Update to TensorFlow 2.2.0 and TensorFlow-Addons 0.10.0.
- CohenKappa metric and WeightedKappaLoose are used as implemented in TensorFlow Addons 0.10.0.

In [3]:
import sys
print('Python {}'.format(sys.version))

Python 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]


Check TensorFlow and TensorFlow-Addons version.

In [4]:
"""
%%time
!pip install /kaggle/input/panda-pp/pygame-1.9.6-cp37-cp37m-manylinux1_x86_64.whl
!pip install --quiet /kaggle/input/keras-applications/Keras_Applications-1.0.8-py3-none-any.whl
!pip install --quiet /kaggle/input/efficientnetrepo110/efficientnet-1.1.0-py3-none-any.whl
"""

'\n%%time\n!pip install /kaggle/input/panda-pp/pygame-1.9.6-cp37-cp37m-manylinux1_x86_64.whl\n!pip install --quiet /kaggle/input/keras-applications/Keras_Applications-1.0.8-py3-none-any.whl\n!pip install --quiet /kaggle/input/efficientnetrepo110/efficientnet-1.1.0-py3-none-any.whl\n'

In [5]:
%%time
!pip install --quiet efficientnet
!pip install --quiet pygame
!pip install --quiet memory_profiler
!pip install --quiet tensorflow-addons==0.10.0

CPU times: user 53.3 ms, sys: 23.9 ms, total: 77.2 ms
Wall time: 13.9 s


In [6]:
import tensorflow as tf
import tensorflow_addons as tfa
print('TensorFlow version:        {}'.format(tf.__version__))
print('TensorFlow-Addons version: {}'.format(tfa.__version__))

TensorFlow version:        2.2.0
TensorFlow-Addons version: 0.10.0


Load essential modules. 

In [7]:
import numpy as np
import pandas as pd
import tensorflow.keras as K
import matplotlib.pyplot as plt
import math

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from tensorflow.keras.utils import Sequence
import albumentations as albu      # a fast image augmentation library

import skimage.io
import json

from tensorflow.keras import Model, Sequential
import efficientnet.tfkeras as efn

import os
import memory_profiler
import time
import gc

import zipfile
from tqdm.notebook import tqdm    # A Fast, Extensible Progress Bar 
                                  # for Python and CLI 
import cv2
import binascii
import io
import pygame                     # a Python wrapper module for the SDL multimedia library

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


Check GPUs availability. 

In [8]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    logical_gpus = tf.config.list_logical_devices('GPU')
    print('{} Physical GPUs, {} Logical GPUs'.format(len(gpus), len(logical_gpus)))

1 Physical GPUs, 1 Logical GPUs


In [9]:
cpus = tf.config.list_physical_devices('CPU')
if cpus:
    logical_cpus = tf.config.list_logical_devices('CPU')
    print('{} Physical CPUs, {} Logical CPUs'.format(len(cpus), len(logical_cpus)))

1 Physical CPUs, 1 Logical CPUs


In [10]:
time_start = time.time()

## Tiles of image

Load whole slide image(WSI), and select 16 128x128 tiles from each image according to the number of tissure pixels.

In [11]:
def tile(img_id, mode = 'train', mask = False, tile_size = 128, num_tile = 16, aug = None):
    # This function selects <num_tile> tiles of size <tile_size>x<tile_size> 
    # for image of <img_id> (and mask) 
    # based on the maximum number of tissue pixels.
    if(mode == 'train'):
        img = skimage.io.MultiImage(os.path.join(TRAIN_PATH, img_id + '.tiff'))[-1] 
    elif(mode == 'test'):
        img = skimage.io.MultiImage(os.path.join(TEST_PATH, img_id + '.tiff'))[-1]
    else:
        raise AttributeError('tile mode Error')
    if aug:
        img = aug(image=img)['image']
    shape = img.shape
    pad0, pad1 = (tile_size - shape[0]%tile_size)%tile_size, (tile_size - shape[1]%tile_size)%tile_size
    img = np.pad(img, [[pad0//2, pad0-pad0//2], [pad1//2, pad1-pad1//2], [0,0]],
                 constant_values = 255)
    img = img.reshape(img.shape[0]//tile_size, tile_size, img.shape[1]//tile_size, tile_size,3)
    img = img.transpose(0,2,1,3,4).reshape(-1, tile_size, tile_size, 3)
    if len(img) < num_tile:
        img = np.pad(img, [[0, num_tile - len(img)], [0, 0], [0, 0], [0, 0]], 
                     constant_values = 255)    
    idxs = np.argsort(img.reshape(img.shape[0], -1).sum(-1))[:num_tile]
    img = np.array(img[idxs])
    
    if(mask):
        mask = skimage.io.MultiImage(os.path.join(MASKS_PATH, img_id + '_mask.tiff'))[-1]
        if aug: 
            mask = aug(image=mask)['image']
        if(mask.shape != shape):
            print(f'ID {img_id} image shape {shape} mask shape {mask.shape}')
            mask = np.zeros_like(img)
            return img, mask
        mask = np.pad(mask, [[pad0//2, pad0-pad0//2], [pad1//2, pad1-pad1//2], [0,0]],
                      constant_values = 0)
        mask = mask.reshape(mask.shape[0]//tile_size, tile_size, mask.shape[1]//tile_size, tile_size,3)
        mask = mask.transpose(0,2,1,3,4).reshape(-1, tile_size, tile_size, 3)
        if len(mask) < num_tile:
            mask = np.pad(mask, [[0, num_tile - len(mask)], [0, 0], [0, 0], [0, 0]], 
                          constant_values = 0)
        mask = np.array(mask[idxs])
        return img, mask
    else:
        return img
    
def glue_to_one(tile_seq):
    l_tile = int(math.sqrt(NUM_TILE))
    img_glue = np.zeros((l_tile*TILE_SIZE, l_tile*TILE_SIZE, 3),
                         dtype = np.float32)
    for i, t in enumerate(tile_seq):
        x = i//l_tile
        y = i%l_tile
        img_glue[x*TILE_SIZE:(x+1)*TILE_SIZE,
                 y*TILE_SIZE:(y+1)*TILE_SIZE, :] = t
    return img_glue

In [12]:
class PANDA_Sequence(Sequence):
    def __init__(self, df, batch_size=16, mode='fit', shuffle = False, aug = None, 
                 num_tile = 16, tile_size = 128, n_classes=6):
        self.df = df            # data frame with the image_id
        self.batch_size = batch_size
        self.mode = mode
        self.shuffle = shuffle
        self.aug = aug
        self.tile_size = tile_size
        self.num_tile = num_tile
        self.n_classes = n_classes
        self.l_tile = int(math.sqrt(self.num_tile))
        if(self.mode == 'fit'):
            self.tile_mode = 'train'
            self.tile_mask = False
        elif(self.mode == 'validate'):
            self.tile_mode = 'train'
            self.tile_mask = False
        elif(self.mode == 'predict'):
            self.tile_mode = 'test'
            self.tile_mask = False
        else:
            raise AttributeError('Sequence mode Error')
        self.on_epoch_end()
    def __len__(self):
        return math.floor(len(self.df) / self.batch_size)
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    def __getitem__(self, index):
        X = np.zeros((self.batch_size, self.l_tile*self.tile_size, self.l_tile*self.tile_size, 3), dtype = np.float32)
        img_batch = self.df[index*self.batch_size : (index+1)*self.batch_size]['image_id'].values
        for i, img_id in enumerate(img_batch):
            img_tiles = tile(img_id, mode = self.tile_mode, mask = self.tile_mask, tile_size = self.tile_size,
                             num_tile = self.num_tile, aug = self.aug)
            X[i,] = glue_to_one(img_tiles)/255.
        if self.mode in ['fit', 'validate']:
            y = np.zeros((self.batch_size, self.n_classes), dtype = np.float32)
            # encode label list
            lbls_batch = self.df[index * self.batch_size: (index+1) * self.batch_size]['isup_grade'].values
            for i in range(self.batch_size):
                y[i, lbls_batch[i]] = 1
            return X, y
        elif self.mode == 'predict':
            return X
        else:
            raise AttributeError('mode parameter error')        

In [13]:
class PANDA_tile_Sequence(Sequence):
    def __init__(self, df, batch_size=16, mode='fit', shuffle = False, aug = None, 
                 num_tile = 16, tile_size = 128, n_classes=6):
        self.df = df            # data frame with the image_id
        self.batch_size = batch_size
        self.mode = mode
        self.shuffle = shuffle
        self.aug = aug
        self.tile_size = tile_size
        self.num_tile = num_tile
        self.n_classes = n_classes
        self.l_tile = int(math.sqrt(self.num_tile))
        if(self.mode == 'fit'):
            self.tile_mode = 'train'
            self.tile_mask = False
        elif(self.mode == 'validate'):
            self.tile_mode = 'train'
            self.tile_mask = False
        elif(self.mode == 'predict'):
            self.tile_mode = 'test'
            self.tile_mask = False
        else:
            raise AttributeError('Sequence mode Error')
        self.on_epoch_end()
    def __len__(self):
        return math.floor(len(self.df) / self.batch_size)
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    def __getitem__(self, index):
        X = np.zeros((self.batch_size, self.l_tile*self.tile_size, self.l_tile*self.tile_size, 3), dtype = np.float32)
        img_batch = self.df[index*self.batch_size : (index+1)*self.batch_size]['image_id'].values
        if self.tile_mode == 'train':
            archive_img = zipfile.ZipFile(ZIP_TRAIN, 'r')
            for i, img_id in enumerate(img_batch):
                data_img = archive_img.read(img_id + '_tiles.png')
                bytes_io = io.BytesIO(data_img)
                img_tiles = pygame.image.load(bytes_io)
                X[i,] = pygame.surfarray.array3d(img_tiles)/255.
        else:
            for i, img_id in enumerate(img_batch):
                img_tiles = tile(img_id, mode = self.tile_mode, mask = self.tile_mask, tile_size = self.tile_size,
                                 num_tile = self.num_tile, aug = self.aug)
                X[i,] = glue_to_one(img_tiles)/255.
        if self.mode in ['fit', 'validate']:
            y = np.zeros((self.batch_size, self.n_classes), dtype = np.float32)
            # encode label list
            lbls_batch = self.df[index * self.batch_size: (index+1) * self.batch_size]['isup_grade'].values
            for i in range(self.batch_size):
                y[i, lbls_batch[i]] = 1
            return X, y
        elif self.mode == 'predict':
            return X
        else:
            raise AttributeError('mode parameter error')        

## Load data

In [14]:
print('Memory usage : {} MB'.format(*memory_profiler.memory_usage(-1)))

Memory usage : 958.59765625 MB


In [15]:
df_train = pd.read_csv('{}/train.csv'.format(DATA_PATH))
print('train: {}'.format(df_train.shape))
print('unique isup grade: {}'.format(df_train['isup_grade'].nunique()))

train: (10616, 4)
unique isup grade: 6


In [21]:
img_mismatch_list =(
        '0bc1fdd9a51cf38d3dffac3541189c02',
        '0c0041c562f7cbcb69a4d195d5a06114',
        '0e9219ba3378fbf1de1743a53f3a0c18',
        '107694232cb071255848ddca5efbe061',
        '184b98556b06d9962dee7e737b23902b',
        '1abf1e5a8a5aef4100c97ffd8ea09ce4',
        '35a01018982323a647399b88ef4d2287',
        '5a4ec1ea101e790664aeab41d54cbe75',
        '7df260d73a124534f81a62768190ba8b',
        '852768d64622d6bb9e0d2c3ef45660f5',
        'a38edfd7f0ab8eb00995e7c1c3168b60',
        'a59b754e6ae29ba224e7422995ebb5fe',
        'b7f0f181cd947530a0d7ecb9c35b992f',
        'b8f8ccffefda42fbe46878921e9d1194',
        'bb33e9befff93c6918e691486eed3990',
        'c2265a32286d90695745e411d313b694'
)
keep_item = df_train['image_id'].apply(lambda x: x not in img_mismatch_list)
df_train = df_train[keep_item]
print('train: {}'.format(df_train.shape))

train: (10600, 4)


In [16]:
# Check the tiles png files
def plot_tiles_png(img_id):
    img = skimage.io.MultiImage(os.path.join(TRAIN_PATH, img_id + '.tiff'))[-1]
    
    archive_img = zipfile.ZipFile(ZIP_TRAIN, 'r')
    data_img = archive_img.read(img_id + '_tiles.png')
    bytes_io = io.BytesIO(data_img)
    img_tiles = pygame.image.load(bytes_io)
    img_tiles = pygame.surfarray.array3d(img_tiles)
    print(img_tiles.shape)

    archive_mask = zipfile.ZipFile(ZIP_MASKS, 'r')
    data_mask = archive_mask.read(img_id + '_mask_tiles.png')
    bytes_io = io.BytesIO(data_mask)
    mask_tiles = pygame.image.load(bytes_io)
    mask_tiles = pygame.surfarray.array2d(mask_tiles)
    print(mask_tiles.shape)
        
    fig, ax = plt.subplots(1, 3, figsize=(18,12))
    fig.suptitle('Image ID: {}  data_provider: {}  ISUP grade: {}'.format(img_id, 
                                 df_train[df_train['image_id']==img_id]['data_provider'].values[0],
                                 df_train[df_train['image_id']==img_id]['isup_grade'].values[0]))
    ax[0].imshow(img)
    ax[1].imshow(img_tiles)
    ax[2].imshow(mask_tiles, cmap = 'hot', vmin = 0, vmax = 5)
    for i in range(3):
        ax[i].axis('off')
    plt.show()
    return
    
#img_id = df_train[df_train['isup_grade']==3]['image_id'].sample(n=1, random_state = SEED).to_numpy()[0]
#plot_tiles_png(img_id)

## Model

Model based on EfficientNetB3:

In [17]:
def get_uncompiled_model():
    conv_base = efn.EfficientNetB3(
        input_shape = (int(math.sqrt(NUM_TILE))*TILE_SIZE, int(math.sqrt(NUM_TILE))*TILE_SIZE, 3),
        weights = 'imagenet', #os.path.join(ENET_MODEL_PATH, 
                  #'efficientnet-b3_weights_tf_dim_ordering_tf_kernels_autoaugment_notop.h5'),  # pretrained weights with ImageNet
        include_top = False,
        pooling = 'avg')
    conv_base = K.Model(inputs=conv_base.inputs, outputs=conv_base.outputs)
    model = Sequential()
    model.add(conv_base)
    #model.add(K.layers.Dropout(.2))
    model.add(K.layers.Dense(6, activation='softmax'))
    conv_base.trainable = False
    
    if restart:
        #model.load_weights(restart_path, by_name = True, skip_mismatch = True)
        #print('model weights loaded')
        # set more layers trainable
        conv_base.trainable = True
        set_trainable = False
        for layer in conv_base.layers:
            if layer.name == 'block7b_add':
                set_trainable = True
            if set_trainable:
                layer.trainable = True
            else:
                layer.trainable = False
        model.load_weights(restart_path, by_name = True, skip_mismatch = True)
        print('model weights loaded')
    else:
        print('train from scratch')

    return model

def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(
        optimizer = K.optimizers.Adam(lr=INI_LR),
        loss = tfa.losses.WeightedKappaLoss(num_classes=6, weightage='quadratic'),
        metrics = ['categorical_accuracy', tfa.metrics.CohenKappa(num_classes=6, weightage='quadratic')]
        )
    return model

## Train the model with KFold cross validation

In [18]:
time_elapsed = time.time() - time_start
print('Time elapsed: {:02.0f}:{:02.0f}:{:02.0f} s'.format(time_elapsed//3600,
                                                  time_elapsed%3600//60,
                                                  time_elapsed%60))

Time elapsed: 00:00:03 s


In [24]:
def kfold_estimate(df, n_fold = N_FOLD, stratefied = True):
    if(stratefied):
        kf = StratifiedKFold(n_splits = n_fold, random_state = SEED, shuffle = True)
    else:
        kf = KFold(n_splits = n_fold, random_state = SEED, shuffle = True)
    loss_per_fold = []
    acc_per_fold = []
    kappa_per_fold = []
    for i_fold, (train_index, val_index) in enumerate(kf.split(df[['image_id']], df['isup_grade'])):
        if(i_fold == 0):  
            loss_per_fold.append( -0.7407200336456299)
            acc_per_fold.append(0.29909366369247437)
            kappa_per_fold.append(0.4803186058998108)
            continue
        # 
        print('-------------------------------------------------------------------')
        print('Training for fold {} ...'.format(i_fold))
        time_kstart = time.time()
        model = get_compiled_model()
        X_train,  X_val = df.iloc[train_index], df.iloc[val_index]
        lbl_value_counts = X_train['isup_grade'].value_counts()
        class_weights = {i: max(lbl_value_counts)/v for i, v in lbl_value_counts.items()}
        print('classes weights: {}'.format(class_weights))

        train_gen = PANDA_tile_Sequence(
            df = X_train,
            batch_size = BATCH_SIZE,
            mode = 'fit',
            shuffle = False,
            aug = None,
            num_tile = NUM_TILE,
            tile_size = TILE_SIZE,
            n_classes = 6)

        val_gen = PANDA_tile_Sequence(
            df = X_val,
            batch_size = BATCH_SIZE,
            mode = 'validate',
            shuffle = False,
            aug = None,
            num_tile = NUM_TILE,
            tile_size = TILE_SIZE,
            n_classes = 6)

        model_file = '{}/model_{}_kf{}.h5'.format(MODEL_PATH, MODEL_VERSION, i_fold)
        earlystopper = K.callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = 10,
            verbose = 1,
            mode = 'min'
            )
        modelsaver = K.callbacks.ModelCheckpoint(
            model_file,
            monitor = 'val_loss',
            verbose = 1,
            save_weights_only = True,
            save_best_only = True,
            mode = 'min'
            )
        lrreducer = K.callbacks.ReduceLROnPlateau(
            monitor = 'val_loss',
            factor = .1,
            patience = 5,
            verbose = 1,
            min_lr = 1.e-7
            )

        history = model.fit(
            train_gen,
            validation_data = val_gen,
            class_weight = class_weights,
            callbacks = [earlystopper, modelsaver, lrreducer],
            epochs = EPOCHS,
            verbose = 1
            )

        loss_per_fold.append(history.history['val_loss'][-1])
        acc_per_fold.append(history.history['val_categorical_accuracy'][-1])
        kappa_per_fold.append(history.history['val_cohen_kappa'][-1])
        print('Score for fold {}:'.format(i_fold))
        print('  loss  {};'.format(loss_per_fold[-1]))
        print('  acc   {};'.format(acc_per_fold[-1]))
        print('  kappa {}.'.format(kappa_per_fold[-1]))
        time_elapsed = time.time() - time_kstart
        print('Time elapsed for fold {:4d}: {:02.0f}:{:02.0f}:{:02.0f}'.format(
            i_fold, time_elapsed//3600, time_elapsed%3600//60, time_elapsed%60))
        mem_usage = memory_profiler.memory_usage(-1)
        format_str = 'Memory usage:            ' + '{:8.0f}'*len(mem_usage) + ' MB'
        print(format_str.format(*mem_usage))
        print('-------------------------------------------------------------------')
        del model, X_train, X_val, history
        gc.collect()
    # Print statistic metrics
    print('-------------------------------------------------------------------')
    print('Score per fold:')
    print('Fold        Loss         Categorical_accuracy            QWK       ')
    print('-------------------------------------------------------------------')
    for i_fold in range(len(loss_per_fold)):
        print('Fold {:4d}    {:12.4f}      {:12.4f}        {:12.4f}'.format(
        i_fold, loss_per_fold[i_fold], acc_per_fold[i_fold], kappa_per_fold[i_fold] 
        ))
    print('-------------------------------------------------------------------')
    print('Statistical metrics:')
    print('Average Loss: {}'.format(np.mean(loss_per_fold)))
    print('Accuracy    : {} (+- {})'.format(np.mean(acc_per_fold), np.std(acc_per_fold)))
    print('QWK         : {} (+- {})'.format(np.mean(kappa_per_fold), np.std(kappa_per_fold)))
    print('-------------------------------------------------------------------')
    return

In [25]:
kfold_estimate(df_train)
mem_usage = memory_profiler.memory_usage(-1)
format_str = 'Memory usage: ' + '{:8.0f}'*len(mem_usage) + ' MB'
print(format_str.format(*mem_usage))        

-------------------------------------------------------------------
Training for fold 1 ...
model weights loaded
classes weights: {0: 1.0, 1: 1.0851703406813626, 2: 2.1552238805970148, 4: 2.3165775401069517, 3: 2.326530612244898, 5: 2.362050163576881}
1987/1987 [==============================] - ETA: 0s - loss: -0.9618 - categorical_accuracy: 0.3037 - cohen_kappa: 0.3846
Epoch 00001: val_loss improved from inf to -0.73437, saving model to /content/drive/My Drive/Colab Notebooks/competetions/PANDA//model_v1.2_kf1.h5
1987/1987 [==============================] - 620s 312ms/step - loss: -0.9618 - categorical_accuracy: 0.3037 - cohen_kappa: 0.3846 - val_loss: -0.7344 - val_categorical_accuracy: 0.3527 - val_cohen_kappa: 0.4847 - lr: 1.0000e-04
Score for fold 1:
  loss  -0.7343723773956299;
  acc   0.3527190387248993;
  kappa 0.48471683263778687.
Time elapsed for fold    1: 00:10:41
Memory usage:         2490 MB
-------------------------------------------------------------------
------------

## Inference

In [ ]:
def train_fullset(df):
    model = get_compiled_model()
    
    X_train = df
    lbl_value_counts = X_train['isup_grade'].value_counts()
    class_weights = {i: max(lbl_value_counts)/v for i, v in lbl_value_counts.items()}
    print('classes weights: {}'.format(class_weights))

    train_gen = PANDA_tile_Sequence(
        df = X_train,
        batch_size = BATCH_SIZE,
        mode = 'fit',
        shuffle = False,
        aug = None,
        num_tile = NUM_TILE,
        tile_size = TILE_SIZE,
        n_classes = 6)

    model_file = '{}/model_{}.h5'.format(MODEL_PATH, MODEL_VERSION)
    earlystopper = K.callbacks.EarlyStopping(
        monitor = 'train_loss',
        patience = 10,
        verbose = 1,
        mode = 'min'
        )
    modelsaver = K.callbacks.ModelCheckpoint(
        model_file,
        monitor = 'train_loss',
        verbose = 1,
        save_weights_only = True,
        save_best_only = True,
        mode = 'min'
            )
    lrreducer = K.callbacks.ReduceLROnPlateau(
        monitor = 'train_loss',
        factor = .1,
        patience = 5,
        verbose = 1,
        min_lr = 1.e-7
        )

    history = model.fit(
        train_gen,
        validation_data = None,
        class_weight = class_weights,
        callbacks = [earlystopper, modelsaver, lrreducer],
        epochs = EPOCHS,
        verbose = 1
        )

    history_file = 'history_{}.txt'.format(MODEL_VERSION)
    dict_to_save = {}
    for k, v in history.history.items():
        dict_to_save.update({
            k: [np.format_float_positional(x) for x in history.history[k]]
        })
    with open(history_file, 'w') as file:
        json.dump(dict_to_save, file)
    ep_max = len(history.history['loss'])
    plt.plot(history.history['loss'][:ep_max], label='loss')
    plt.legend()
    plt.show()
    plt.plot(history.history['categorical_accuracy'][:ep_max], label='accuracy')
    plt.plot(history.history['cohen_kappa'][:ep_max], label = 'cohen_kappa')
    plt.legend()
    plt.show()
    del X_train, history
    return model

In [ ]:
def load_model():
    model = get_compiled_model()
    return model

In [ ]:
def make_submit():
    df_test = pd.read_csv('{}/test.csv'.format(DATA_PATH))
    print(df_test.shape)
    pred = np.zeros((len(df_test), 6))
    if os.path.exists(TEST_PATH):
        sub_gen = PANDA_Sequence(
            df = df_test,
            batch_size = 1,
            mode = 'predict',
            shuffle = False,
            aug = None,
            num_tile = NUM_TILE,
            tile_size = TILE_SIZE,
            n_classes = 6
        )
        pred = model.predict(sub_gen)
        print('Predict for {} images'.format(len(pred)))
    else:
        print('Predict zeros')

    df_test['isup_grade'] = np.argmax(pred, axis = 1)
    df_test.drop('data_provider', axis = 1, inplace = True)
    df_test.to_csv('submission.csv', index = False)
    print('submission saved')
    return

In [ ]:
%%time
#model = train_fullset(df_train)

In [ ]:
%%time
#model = load_model()
#make_submit()

In [ ]:
!head /kaggle/working/submission.csv